In [2]:
import tensorflow as tf
import numpy as np
import random as pr
import matplotlib.pyplot as plt
from gym.envs.registration import register
#import gym.envs.board_game
import gym

In [4]:
gym.make('SpaceInvaders-v0')

[2017-09-14 11:06:54,468] Making new env: SpaceInvaders-v0


OSError: /home/shephexd/anaconda3/lib/python3.6/site-packages/zmq/backend/cython/../../../../.././libstdc++.so.6: version `GLIBCXX_3.4.20' not found (required by /home/shephexd/anaconda3/lib/python3.6/site-packages/atari_py/ale_interface/build/libale_c.so)

In [ ]:
from collections import deque
from IPython import display

In [ ]:
env = gym.make('CartPole-v0')

input_size = env.observation_space.shape[0]
output_size = env.action_space.n

MAX_REPLAY_MEMORY = 50000
dis = 0.9

In [ ]:
class DQN(object):
    def __init__(self, session, input_size, output_size, name="main"):
        self.session = session
        self.input_size = input_size
        self.output_size = output_size
        self.net_name = name
        
        self._build_network()
    
    def _build_network(self, h_size=10, learning_rate=1e-1):
        with tf.variable_scope(self.net_name):
            self._X = tf.placeholder(
            tf.float32, [None, self.input_size], name="input_x")
            
            W1 = tf.get_variable("W1", shape=[self.input_size, h_size],
                                initializer=tf.contrib.layers.xavier_initializer())
            layer1 = tf.nn.tanh(tf.matmul(self._X, W1))
            
            W2 = tf.get_variable("W2", shape=[h_size, self.output_size],
                                initializer=tf.contrib.layers.xavier_initializer())
            self._Qpred = tf.matmul(layer1, W2)
            
        self._Y = tf.placeholder(
        shape=[None, self.output_size], dtype=tf.float32)
        
        self._loss = tf.reduce_mean(tf.square(self._Y - self._Qpred))
        
        self._train = tf.train.AdamOptimizer(
        learning_rate=learning_rate).minimize(self._loss)
        
    def predict(self, state):
        x = np.reshape(state, [1, self.input_size])
        return self.session.run(self._Qpred, feed_dict={self._X: x})
    
    def update(self, x_stack, y_stack):
        return self.session.run([self._loss, self._train], feed_dict={
            self._X:x_stack, self._Y:y_stack
        })

In [ ]:
def simple_replay_train(DQN, train_batch):
    x_stack = np.empty(0).reshape(0, DQN.input_size)
    y_stack = np.empty(0).reshape(0, DQN.output_size)
    
    for state, action, reward, next_state, done in train_batch:
        Q = DQN.predict(state)
        
        if done:
            Q[0, action] = reward
        else:
            Q[0, action] = reward + dis * np.max(DQN.predict(next_state))
        y_stack = np.vstack([y_stack, Q])
        y_stack = np.vstack([x_stack, state])
    
    return DQN.update(x_stack, y_stack)

In [ ]:
def show_state(env, step=0, info=""):
    plt.figure(3)
    plt.clf()
    plt.imshow(env.render(mode='rgb_array'))
    plt.title("%s | Step: %d %s" % (env._spec.id,step, info))
    plt.axis('off')

    display.clear_output(wait=True)
    display.display(plt.gcf())

In [ ]:
%matplotlib inline
render = lambda : plt.imshow(env.render(mode='rgb_array'))

def bot_play(mainDQN):
    s = env.reset()
    reward_sum = 0
    while True:
        env.reset()
        render()
        a = np.argmax(mainDQN.predict(s))
        s, reward, done, _ = env.step(a)
        reward_sum += reward
        if done:
            print("Total score: {}".format(reward_sum))
            break

In [ ]:
def main():
    max_episodes = 5000
    replay_buffer = deque()
    
    with tf.Session() as sess:
        mainDQN = DQN(sess, input_size, output_size)
        tf.global_variables_initializer().run()
        
        for episode in range(max_episodes):
            e = 1. / ((episode / 10) + 1)
            done = False
            step_count = 0
            
            state = env.reset()
            
            while not done:
                if np.random.rand(1) < e:
                    action = env.action_space.sample()
                else:
                    action = np.argmax(mainDQN.predict(state))
                
                next_state, reward, done, _ = env.step(action)
                
                if done:
                    reward = -100
                
                #Save the experience to our buffr
                replay_buffer.append((state, action, reward, next_state, done))

                if len(replay_buffer) > MAX_REPLAY_MEMORY:
                    replay_buffer.popleft()
                    
                state = next_state
                step_count += 1
                if step_count > 100000:
                    break
                print("Episode: {} steps: {}".format(episode, step_count))
                if step_count > 100000:
                    pass

                if episode % 10 == 1:
                    for _ in range(50):
                        minibatch = pr.sample(replay_buffer, 10)
                        loss, _ = simple_replay_train(mainDQN, minibatch)
                    print("Loss: ", loss)
            bot_play(mainDQN)

In [ ]:
%matplotlib inline

main()

In [8]:
# The typical imports
import gym
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Imports specifically so we can render outputs in Jupyter.
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython.display import display


def display_frames_as_gif(frames):
    """
    Displays a list of frames as a gif, with controls
    """
    #plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 72)
    patch = plt.imshow(frames[0])
    plt.axis('off')

    def animate(i):
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), animate, frames = len(frames), interval=50)
    display(display_animation(anim, default_mode='loop'))

In [9]:
env = gym.make('CartPole-v0')

# Run a demo of the environment
observation = env.reset()
cum_reward = 0
frames = []
for t in range(5000):
    # Render into buffer. 
    frames.append(env.render(mode = 'rgb_array'))
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
    if done:
        break
env.render(close=True)
display_frames_as_gif(frames)

[2017-09-07 22:10:16,071] Making new env: CartPole-v0


NameError: name 'base' is not defined